<a href="https://colab.research.google.com/github/jkevorkian/diabetox/blob/main/diabetox.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Exploracion Dataset/API